In [2]:

# OPTIMIZACION SMA 

import yfinance as yf
import pandas as pd 
import numpy as np
from scipy.signal import filtfilt, freqz, butter
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.stats import t


# Simple Moving Averge
def SMA(close,T):
    return pd.Series(close).rolling(T).mean()

# Bollinger Bands 
def Vol_L(close,T,smooth,num_std = 2):
    return smooth - num_std*pd.Series(close).rolling(T).std()

def Vol_U(close,T,smooth,num_std = 2):
    return smooth + num_std*pd.Series(close).rolling(T).std()

In [3]:
# ESTRATEGIA SMA

class strat_SMA(Strategy):

    T = 18 # period SMA
    N = 2  
    sl_pct = 0.015

    # Un butterworth de Wc = 2/T tiene la misma Wc que un SM de T periodos. Pero la SMA cambia. 
    # Aunque se podria argumentar que el SM tiene la misma Wc que la SMA y por tanto la SMA tiene la misma Wc que el BUTT. 

    def init(self):

        # Parameters 
        close = self.data.Close
        self.smooth = SMA(close,self.T)

        # Indicators 
        self.upper_band = self.I(Vol_U, close, self.T, self.smooth, self.N)
        self.lower_band = self.I(Vol_L, close, self.T, self.smooth, self.N)

    def next(self):

        price = self.data.Close[-1]

        # Buy   
        if crossover(self.lower_band,self.data.Close):
            self.position.close()
            self.buy(sl = price*(1-self.sl_pct))
            
        # Sell
        elif crossover(self.data.Close,self.upper_band):
            self.position.close()
            self.sell(sl = price*(1+self.sl_pct))

In [ ]:
# DATA IMPORT AND BACKTESTING 

xls = r'C:\Users\Usuario\Desktop\TFG\S&P 500 Companies (Standard and Poor 500).xlsx'
file_df = pd.read_excel(xls)
all_tickers = file_df['Symbol'][0:6]
tickers = all_tickers; w = 1/len(tickers)

mu = []; sigma = []; rets = []; ntrades = []; eq = 0

for i in range(len(tickers)):
    try:
        df = yf.download(tickers[i],  start = '2000-01-01', end = '2023-01-01', progress=False)
        iCash = 10000
        com = 0.00

        #BACKTESTING 
        bt = Backtest(df, strat_SMA, cash = iCash, commission = com, exclusive_orders = True)

        # OUTPUTS
        output = bt.run()
        rets.append(output['# Trades'])
        mu.append(output['Return (Ann.) [%]'])
        sigma.append(output['Volatility (Ann.) [%]'])
        ntrades.append(output['# Trades'])

    except: 
        rets.append(None); mu.append(None); sigma.append(None); ntrades.append(None)

# RESULTS
res = pd.DataFrame({'Ticker':tickers, 'Annualized Returns (%)':mu, 'Annualized Volatility (%)':sigma, 'Trades':ntrades}).dropna()
res = res[res['Trades'] != 0]
res = res[res['Annualized Volatility (%)'] < 200]
mean_values = res.mean()
mean_row = pd.DataFrame(mean_values).T
res = pd.concat([res, mean_row]).reset_index(drop=True)
res.loc[res.index[-1], 'Ticker'] = 'MEAN'

#bt.plot(plot_volume = False)